In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from queue import PriorityQueue as pq
import editdistance
from nltk.corpus import words
import cohere

In [38]:
adv_reads = pd.read_csv('google_books_1299.csv')
adv_reads.drop(columns=adv_reads.columns[0], axis=1, inplace=True)

In [39]:
adv_reads.drop_duplicates(keep='first', inplace=True, ignore_index=True)

# manually set: should we do this?
adv_reads['lower_age'] = [16 for i in range(len(adv_reads))]
adv_reads['upper_age'] = [20 for i in range(len(adv_reads))]
# adv_reads.head(2000)


In [40]:
child_books = pd.read_csv("children_books.csv")
child_books.drop_duplicates(keep='first', inplace=True, ignore_index=True)
# child_books.head(5000)

In [5]:
# we want to replace reading age with appr. age lim
# new columns: upper_age, lower_age
upper_age = []
lower_age = []
for ind in child_books.index: 
    # print(child_books['Reading_age'][1][:2])
    if '-' in child_books['Reading_age'][ind]:
        lower, upper = child_books['Reading_age'][ind].split('-')
        upper_age.append(int(upper))
        lower_age.append(int(lower))
    else: # + 
        lower = int(child_books['Reading_age'][ind][:2].strip(string.punctuation))
        upper = int(lower+3) # manually set 
        upper_age.append(upper)
        lower_age.append(lower)

# add age columsn to child_books
child_books['lower_age'] = lower_age
child_books['upper_age'] = upper_age

In [6]:
child_stories = pd.read_csv('children_stories.Csv')
child_stories.drop_duplicates(keep='first', inplace=True, ignore_index=True)
story_upper = []
story_lower = []
# Strip off age and split by -
for ind in child_stories.index:
    age = child_stories['cats'][ind][4:]
    if '-' in age:
        lower, upper = age.split('-')
        story_lower.append(int(lower))
        story_upper.append(int(upper))
    else: #+
        try: 
            lower = int(age.strip(string.punctuation))
            upper = lower + 3
            story_lower.append(int(lower))
            story_upper.append(int(upper))
        except: # in this case we will default to 2-9 years (default)
            story_lower.append(2)
            story_upper.append(9)
            
# Add 2 columns to child_stories
child_stories['lower_age'] = story_lower
child_stories['upper_age'] = story_upper

# child_stories['Author'] = [None for i in range(len(child_stories))]

In [7]:
# child_books.head()
# child_stories.head(2000)
# We won't be using the 'Reading_age' column 

In [8]:
child_books.columns, child_stories.columns

(Index(['Title', 'Author', 'Desc', 'Inerest_age', 'Reading_age', 'lower_age',
        'upper_age'],
       dtype='object'),
 Index(['names', 'cats', 'desc', 'lower_age', 'upper_age'], dtype='object'))

In [9]:
# is there a way to combine these two dataframes together? 
# we only need the title, upperage, lowerage, desc and author from each dataframe
df_1 = child_books.copy().drop(columns = ['Inerest_age', 'Reading_age'])
df_1['Type'] = 'Books'
df_1 = df_1.rename(columns = {'Title': 'title', 'Desc' : 'desc', 'Author':'author'})

df_2 = child_stories.copy().drop(columns = ['cats']) 
df_2['Type'] = 'Stories'
df_2 = df_2.rename(columns = {'names': 'title', 'Author':'author'})

df_combined = pd.concat([df_1, df_2])
# df_combined.value_counts('title')
# df_combined

In [10]:
# Rename columns of adv_reads 
try: 
    adv_reads.drop(columns=['rating', 'voters','price','currency','publisher','page_count','generes','ISBN','language','published_date'], inplace=True)
except:
    pass
adv_reads = adv_reads.rename(columns = {'description': 'desc'})
adv_reads['Type'] = ['Books' for i in range(len(adv_reads))]
adv_reads.fillna('', inplace=True)
# adv_reads

In [11]:
# get rid of numbers
df_combined['title_formatted'] = df_combined['title'].str.replace('\d+', '')
df_combined['desc_formatted'] = df_combined['desc'].str.replace('\d+', '')
adv_reads['title_formatted'] = adv_reads['title'].str.replace('\d+','')
adv_reads['desc_formatted'] = adv_reads['desc'].str.replace('\d+','')

# make lowercase
df_combined['title_formatted'] = df_combined['title_formatted'].apply(str.lower)
df_combined['desc_formatted'] = df_combined['desc_formatted'].apply(str.lower)
adv_reads['title_formatted'] = adv_reads['title_formatted'].apply(str.lower)
adv_reads['desc_formatted'] = adv_reads['desc_formatted'].apply(str.lower)


<ipython-input-11-cecfd9b579cc>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_combined['title_formatted'] = df_combined['title'].str.replace('\d+', '')
<ipython-input-11-cecfd9b579cc>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_combined['desc_formatted'] = df_combined['desc'].str.replace('\d+', '')
<ipython-input-11-cecfd9b579cc>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  adv_reads['title_formatted'] = adv_reads['title'].str.replace('\d+','')
<ipython-input-11-cecfd9b579cc>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  adv_reads['desc_formatted'] = adv_reads['desc'].str.replace('\d+','')


In [12]:
# We can tokenize the words here to remove punctuation 
# Tokenize words 
nltk.download('punkt')
df_combined['title_desc'] = df_combined['title_formatted'].apply(word_tokenize) + df_combined['desc_formatted'].apply(word_tokenize)
adv_reads['title_desc'] = adv_reads['title_formatted'].apply(word_tokenize) + adv_reads['desc_formatted'].apply(word_tokenize)
# df_combined

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
# Remove stopwords
nltk.download('stopwords')
def remove_stopwords(input1):
    words = []
    for word in input1:
        if word not in stopwords.words('english'):
            words.append(word)
    return words

df_combined['title_desc'] =  df_combined['title_desc'].apply(remove_stopwords)
adv_reads['title_desc'] =  adv_reads['title_desc'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
# Lemmatize 
nltk.download('wordnet')
nltk.download('omw-1.4')
lem = WordNetLemmatizer()
def lemma_wordnet(input):
    return [lem.lemmatize(w) for w in input]
df_combined['title_desc'] = df_combined['title_desc'].apply(lemma_wordnet)
adv_reads['title_desc'] = adv_reads['title_desc'].apply(lemma_wordnet)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [15]:
# translate list to string
def join_text(input):
    combined = ' '.join(input)
    return combined
df_combined['title_desc_filtered']=df_combined['title_desc'].apply(join_text)
adv_reads['title_desc_filtered']=adv_reads['title_desc'].apply(join_text)
# adv_reads

In [17]:
# Remove punctuation for clarity 
df_combined['title_desc_filtered'] = df_combined['title_desc_filtered'].str.replace(r'[^\w\s]+', '')
# df_combined

<ipython-input-17-1c4badf68a31>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_combined['title_desc_filtered'] = df_combined['title_desc_filtered'].str.replace(r'[^\w\s]+', '')


In [18]:
user_age = '17'

In [19]:
user_age = int(user_age)
if int(user_age) >= 16: 
    df_16_plus = pd.concat([df_combined, adv_reads])
    df_filter = df_16_plus[(df_16_plus['lower_age'] <= user_age) & (df_16_plus['upper_age'] >= user_age)]
else: 
    df_filter = df_combined[(df_combined['lower_age']<=user_age) & (df_combined['upper_age']>=user_age) ]
# df_filter

In [20]:
vectorizer = TfidfVectorizer(max_features=1800, lowercase=True, stop_words='english', ngram_range=(1,1)) #unigrams and bigrams
tf_idf_output = vectorizer.fit_transform(df_filter['title_desc_filtered'])
vocab = np.array(vectorizer.get_feature_names())
# vocab

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [21]:
num_topics = '25'

In [22]:
# Perform LDA 
num_topics = int(num_topics) # can be changed 
lda = LatentDirichletAllocation(n_components=num_topics, random_state=1).fit(tf_idf_output)

In [23]:
num_top_words = '15'

In [24]:
num_top_words = int(num_top_words) # can be changed. Can also be generated from Cohere
  
topics_set = []

# draft that works
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        topics_set.append(set([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lda, vocab, num_top_words)

In [25]:
user_phrase = 'small town murder that results in a court case from the victim'

In [26]:
# word correct
nltk.download('words')
words_corpus = words.words()

user_input_list = user_phrase.split()
user_input_list = [elem.strip(string.punctuation).lower() for elem in user_input_list]

def run_autocorrect(input_word): 
    closest_words = pq()
    for word in words_corpus: 
        distance = editdistance.eval(word, input_word)
        closest_words.put((distance, word))
    return closest_words.get()

new_phrase_list = []
for elem in user_input_list: 
    if elem not in words_corpus: 
        correct_word = run_autocorrect(elem)
        new_phrase_list.append(correct_word[1])
    else: 
        new_phrase_list.append(elem)

user_phrase = " ".join(new_phrase_list)
user_phrase

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


'small town murder that result in a court case from the victim'

In [27]:
# There is no point of removing stopwords here because we are seeing how much similarity there is regardless. 
# I am thinking of adding spell-correction here? 

# user_phrase = set(user_phrase.lower().split())
# we will preprocess the user phrase just like how we preprocessed the descriptions. Removing stopwords won't matter
# since we are looking at frequency 

user_phrase = word_tokenize(user_phrase)
user_phrase = [word if word not in string.punctuation else '' for word in user_phrase]
 #user_phrase.replace(string.punctuation, '')
user_phrase = set([lem.lemmatize(w).lower() for w in user_phrase])
# print(user_phrase)

best_topic = topics_set[0] #initialization 
best_intersection = -1
for group in topics_set: 
    # print(group, user_phrase, len(user_phrase.intersection(group)))
    print(group)
    if len(user_phrase.intersection(group)) > best_intersection: 
        best_topic = group
        best_intersection = len(user_phrase.intersection(group))

print(best_topic, best_intersection) # returns best list of topics and length of the intersection 

{'trilogy', 'big', 'hitchhiker', 'jason', 'arthur', 'galaxy', 'guide', 'elio', 'onestate', 'secret', 'dent', 'edition', 'kinsey', 'adventure', 'universe'}
{'greg', 'winning', 'helen', 'diary', 'talk', 'author', 'day', 'meeny', 'eeny', 'award', 'contemporary', 'jump', 'feminist', 'grace', 'president'}
{'platform', 'le', 'need', 'step', 'trading', 'day', 'robert', 'thing', 'control', 'mental', 'effective', 'game', 'anthony', 'winterfell', 'money'}
{'blomkvist', 'investing', 'salander', 'kat', 'vagrant', 'darksiders', 'war', 'destruction', 'stock', 'shining', 'amy', 'little', 'thrawn', 'market', 'leader'}
{'offering', 'renowned', 'medicine', 'emergency', 'build', 'business', 'tracy', 'marketing', 'brian', 'simple', 'success', 'successful', 'overcome', 'customer', 'personal'}
{'memoir', 'influence', 'gangster', 'persuasion', 'chaos', 'sherlock', 'story', 'witcher', 'map', 'origin', 'holmes', 'weapon', 'edition', 'zeinab', 'force'}
{'regarding', 'sorcerer', 'series', 'fantasy', 'poirot', 'h

In [28]:
# df_filter

In [29]:
# Now we need to match the book descriptions to the topic that was chosen 
books = pq() #min heap 
for index, row in df_filter.iterrows():
    # print(row['title_desc_filtered'])
    intersect_len = len(set(row['title_desc_filtered'].split()).intersection(best_topic))
    # print(set(row['title_desc_filtered'].split()))
    books.put((-1*intersect_len, row['title'].strip(string.punctuation), row['author'].strip(string.punctuation), row['Type'].strip(string.punctuation)))


In [30]:
num_recs = '5'

In [31]:
books_hm = {}
while len(books_hm) < 5: 
    book = books.get()
    # print(book[0])
    books_hm[book[1]] = (book[2], book[3])

books_hm

{'The Red Signal: An Agatha Christie Short Story': ('Agatha Christie',
  'Books'),
 'The Mysterious Affair at Styles': ('Agatha Christie', 'Books'),
 'A Trace of Crime (a Keri Locke Mystery--Book #4': ('Blake Pierce', 'Books'),
 'A Trace of Vice (a Keri Locke Mystery--Book #3': ('Blake Pierce', 'Books'),
 'Death in an Elegant City: An English Village Murder Mystery': ('Sara Rosett',
  'Books')}

In [32]:
# cohere API key
# KTQmgyEWSk81jeyS98xCMB1iRuWjZ5KDzSkrdw0b

In [33]:
theme = 'crime, murder, new york city, suspect, man, FBI, detective'

In [34]:
import cohere
co = cohere.Client('KTQmgyEWSk81jeyS98xCMB1iRuWjZ5KDzSkrdw0b')
prediction = co.generate(
  model='large',
  prompt='--\nProduct: Book \nKeywords: ' + str(theme) + str('Exciting Book Description:'),
  max_tokens=50,
  temperature=0.8,
  k=0,
  p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop_sequences=["--"],
  return_likelihoods='NONE')
print('Idea: {}...'.format(prediction.generations[0].text))
prediction.generations

Idea: 
The killings are increasing with each passing day in Manhattan.
Detective Alicia Berenson and her partner, Detective Michael Norton, are desperate to find the murderer before he strikes again. But they are losing time. The killer has become a...


[cohere.Generation {
 	text: 
 The killings are increasing with each passing day in Manhattan.
 Detective Alicia Berenson and her partner, Detective Michael Norton, are desperate to find the murderer before he strikes again. But they are losing time. The killer has become a
 	likelihood: None
 	token_likelihoods: None
 }]

In [35]:
send_sms = 'True'

In [36]:
from twilio.rest import Client 

if send_sms == str(True): 
    # send to twilio 
    account_sid = 'AC10d95cbc904be6f7f854a20da39496ec' 
    auth_token = 'b7b13b9326b3a9689c53d3a87521bcf5' 
    client = Client(account_sid, auth_token) 
    
    message = client.messages.create(  
                                messaging_service_sid='MG92509adabb70c0340dc5196debb415a4', 
                                body='Here\'s your idea:' + str(idea) + "\nNow it's time for you to write your heart out!",      
                                to='+16462867923'
                            ) 
    
    print(message.sid)

IndentationError: expected an indented block (<ipython-input-36-ea0bf1780628>, line 2)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a000e112-1cbe-4ff9-9e6d-cef9a0ebc108' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>